In [ ]:
# 불러오는 파일 경로들은 사용중인 컴퓨터에 저장된 경로에 따라 수정이 필요합니다. 
# 크롤링 및 기타 자료 수집에 시간을 쓰지 않기 위해선 "0전환.xlsx" 와 "kospiCode.xlsx" 파일이 필요합니다. 
# (JTH_database for Python.zip 압축파일에 있음.)

from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from lxml import html
import re
from itertools import groupby
from collections import Counter
import numpy as np
import xlrd
import html5lib
import datetime

# 크롤링 함수 정의

def Crawling(code):
    
    url = "http://finance.naver.com/item/board.nhn?code={0}&page=1".format(code)
    HEADERS={'user-agent':('Mozilla/5.0 (Windows NT 6.1)' 'AppleWebKit/537.36 (KHTML, like Gecko)' 'Chrome/61.0.3163.100 Safari/537.36')}

    r = requests.get(url, headers=HEADERS)
    soup = bs(r.text,'html.parser').select("tbody")[2]
    try: 
        end_page=int(re.search('page=(.*?)"',str(soup.select('.pgRR'))).group(1))
    except: 
        end_page=1
        
    resultList=[]
    for i in range(1,end_page+1):
        url = "http://finance.naver.com/item/board.nhn?code={0}&page={1}".format(code, i)
        r = requests.get(url)
        soup = bs(r.text,'html.parser')
        search_table = soup.findAll("span", class_="gray03")
        list1=[date.text for date in search_table]
        list2=[s for s in list1 if len(s) > 10]    
        list3=[l[:10].replace('.','-') for l in list2]
        for u in list3:        
            resultList.append(u) 
        counts=dict(Counter(resultList))
        data=pd.Series(counts,name=code)
        data2=data.to_frame()
        print(i)
    return data2

kospi=pd.read_excel('kospiCode.xlsx',header=None, dtype=object) # 경로 설정 및 코스피 종목 파일 필요
kospiCode=kospi[0].tolist()

# 크롤링 함수 실행코드 (한 번에 돌리면 오류 발생 확률이 매우 높으므로 나눠서 돌릴 필요 있음)

appended_data=[]
for i in kospiCode:
    data=Crawling(i)
    appended_data.append(data)
    print("종목코드 "+i+" 리퀘스트완료...")
appended_data=pd.concat(appended_data, axis=1)
print("표 통합완료")

appended_data = appended_data.fillna(0)
appended_data.to_excel('0전환.xlsx')

df = pd.read_excel('0전환.xlsx')

result_dict = dict()
for i in range(0,736):
    company_name = df.columns[i]
    result_dict[company_name] = [list(df[company_name].loc[lambda s: s>df[company_name].mean()+df[company_name].std()*-3].index),\
                                list(df[company_name].loc[lambda s: s>df[company_name].mean()+df[company_name].std()*-2].index),\
                                list(df[company_name].loc[lambda s: s>df[company_name].mean()+df[company_name].std()*2].index),\
                                list(df[company_name].loc[lambda s: s>df[company_name].mean()+df[company_name].std()*3].index)]

new_df = pd.DataFrame(result_dict, index = ["std_-3","std_-2","std_2","std_3"])

new_df.to_excel("표준편차별정리테이블2.xlsx")

df = pd.read_excel("표준편차별정리테이블2.xlsx")

# 이 아래 부분은 '표준편차별정리테이블2.xlsx' 파일에서 std_3, 즉 평균게시글보다 3 표준편차 이상 게시글이 올라온 날짜만을 뽑아내어,
# SAS 에서 돌릴 수 있도록 첫번째 열에는 종목코드, 두번째 열에는 해당 날짜가 들어가도록 엑셀을 만듭니다. 
# 표준편차 2 이상을 구하는 경우 아래 코드에서 "std_3" 라고 적힌 부분을 "std_2" 로 바꾸어 따로 저장했습니다. 

for i in df.columns:
    dict1[i] = ast.literal_eval(df.loc["std_3",i])

df2= pd.DataFrame.from_dict(dict1, orient="index")

series1 = pd.Series()
for i in range(736):
    list1 = list(dict1.values())[i]
    se1 = pd.Series(list1)
    se1.index = [df.columns[i]]*len(list(dict1.values())[i])
    series1 = series1.append(se1)
    
series1.to_excel("SAS용_std3.xlsx")

# 옵션!
# 매일매일 코스피 전체를 크롤링해서, std3 이상 게시글이 올라온 코드를 메일로 전송하기(이후 윈도우 스케쥴러 적용 필요)

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime
import re

def timer_crawling_first(code, num_page):
        
    url = "http://finance.naver.com/item/board.nhn?code={0}&page={1}".format(code, num_page)
    HEADERS={'user-agent':('Mozilla/5.0 (Windows NT 6.1)' 'AppleWebKit/537.36 (KHTML, like Gecko)' 'Chrome/61.0.3163.100 Safari/537.36')}
    r = requests.get(url, headers=HEADERS)
    soup = bs(r.text,'html.parser').select('tbody')[0]
    dates = soup.findAll("span", class_="gray03")
    now = datetime.datetime.now()
    p = re.compile(str(now)[:10].replace('-','.'))
    todaylist = p.findall(str(dates))
    length = len(todaylist)
    return length

def additional(code, num_page):
    
    if timer_crawling_first(code, num_page) < 20 :
        return timer_crawling_first(code, num_page)
    else: 
        return 20 + additional(code, num_page+1)

kospi = pd.read_excel('C:\\Users\\acous\\PythonCoding\\Projects\\kospiCode.xlsx',header=None, dtype=object)
kospiCode = kospi[0].tolist() 

resultdict = {}
for i in kospiCode:
    result = additional(i,1)
    resultdict[i] = result
    
df=pd.read_excel('C:\\Users\\acous\\PythonCoding\\Projects\\0전환.xlsx')
    
mean_plus_std3 = {}
for i in df.columns:
    mean_plus_std3[i] = df[i].mean()+df[i].std()*3
    
def compare(countdict,meandict):
    return [i for i in df.columns if countdict[i] > meandict[i]]   

final = compare(resultdict,mean_plus_std3)
text_contents = str(final)

# 메일보내는용
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def sendMail(from_email,to_email,text_cont,html_cont):

    message = MIMEMultipart('alternative')
    message['Subject'] = "오늘의 종목: 표준편차3 이상 게시글 개수"
    message['From'] = from_email
    message['To'] = to_email

    part1 = MIMEText(text_cont,'plain')
    part2 = MIMEText(html_cont,'html')

    message.attach(part1)
    message.attach(part2)


    smtp_server = smtplib.SMTP('smtp.gmail.com:587')
    smtp_server.starttls()
    smtp_server.login("개인 이메일 주소 입력", "개인 비밀번호 입력")
    smtp_server.sendmail(from_email,to_email,message.as_string())
    smtp_server.quit()      
    
html_contents = str(final)
    
try: 
    sendMail("보내는 메일 주소","받는 메일 주소(보내는 메일과 받는 메일 같게도 가능)",text_contents,html_contents)
except: print('문제가 있다')